# Preparation Notebook

In this notebook we made some preperation operations for our applications. e.g. for DUBUS app we Model training, data indexing bu different feature.

In [2]:
### Imports ###
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator, Normalizer, StringIndexerModel, OneHotEncoderModel, VectorSizeHint

base_path = "/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/"

!pip install libify
import libify
lab4_functions = libify.importer(globals(), '/Users/shubi@campus.technion.ac.il/Lab4_functions')

## Offline Preperations
# %fs rm -r /dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/
# %fs ls /dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il

In [3]:

batch_df = lab4_functions.create_warehouse_df(streaming=False, USE_TAKE=False)
lab4_functions.save_indexer("areaId1", batch_df)
indexer = StringIndexerModel.load(f"{base_path}indexer_areaId1")
batch_df_indexed = indexer.transform(batch_df)
lab4_functions.save_encoder("areaId1Index", batch_df_indexed)

In [4]:

batch_df = lab4_functions.create_warehouse_df(streaming=False, USE_TAKE=False)
lab4_functions.save_indexer("journeyPatternId", batch_df)
indexer = StringIndexerModel.load(f"{base_path}indexer_journeyPatternId")
batch_df_indexed = indexer.transform(batch_df)
lab4_functions.save_encoder("journeyPatternIdIndex", batch_df_indexed)

In [5]:

batch_df = lab4_functions.create_warehouse_df(streaming=False, USE_TAKE=False)
batch_df = batch_df.withColumn('hourGroup', hour_group(F.col('currentHour')).cast("int")).withColumn('day', date_format("date", 'yyyy-MM-dd'))
save_indexer("hourGroup", batch_df)
indexer = StringIndexerModel.load(f"{base_path}indexer_hourGroup")
batch_df_indexed = indexer.transform(batch_df)
lab4_functions.save_encoder("hourGroupIndex", batch_df_indexed)

In [6]:
min_date = '2017-07-03'
max_date = '2018-06-30'

# Prepare Train on batch data from 2017 for model
batch_df_train = lab4_functions.create_warehouse_df(streaming=False, USE_TAKE=False)
batch_df_train = batch_df_train.where(batch_df_train.date.between(min_date, max_date))
hydrate_data_batch_train = lab4_functions.hydrate_data(batch_df_train)
grouped_df_batch_train = lab4_functions.aggregate_data(hydrate_data_batch_train)
input_train_data = lab4_functions.encode_and_assemble(grouped_df_batch_train)
model = lab4_functions.train_model(input_train_data, maxIter = 15)
model.save(f"{base_path}model_new_new")

In [7]:
dbutils.fs.ls("/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il")

Out[1]: [FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/encoder_areaId1Index/', name='encoder_areaId1Index/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/encoder_hourGroupIndex/', name='encoder_hourGroupIndex/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/encoder_journeyPatternIdIndex/', name='encoder_journeyPatternIdIndex/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/indexer_areaId1/', name='indexer_areaId1/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/indexer_hourGroup/', name='indexer_hourGroup/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/indexer_journeyPatternId/', name='indexer_journeyPatternId/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/model/', name='model/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/model_new/', name='model_new/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/model_new_new/', name='model_new_new/', size=0)]

In [8]:

df = lab4_functions.create_warehouse_df(streaming=False, USE_TAKE=False, min_date=None, max_date=None, path_ip='/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile')
if df=='problem':
    print('problem')
df = lab4_functions.hydrate_data(df)
df = lab4_functions.aggregate_data(df, streaming=False)
df = df.drop('congestionPercentage')
lab4_functions.write_to_elastic(df, 'hydrated-grouped-removed-uncertain-df', lab4_functions.elastic_settings_mappings_date, overwrite=True, streaming=False)